# 卷积
* 稀疏连接 让学习参数变得很少
* 平移不变性 不关心物体出现在图像什么位置
* 卷积计算过程
  * ![](images/convolution-2022-09-28-10-49-08.png)
  * 输入特征图 m个通道，宽w,高h
  * 输出有n个特征图,宽h',高w'
  * 卷积核大小kxk，m通道，总共n个卷积核，卷积核核数=输出的特征图的通道数
  * 多通道时卷积
    * 输入特征图的第i个特征图与卷积核中的第i个通道进行卷积，生成m个特征图，后将这m个特征图对应位置求和，
    * m个特征图合并为输出特征图中一个通道的特征图
  * padding
    * 多层卷积时，特征图会一点点变小，可以减缓变小速度，在计算卷积时，对输入特征图补0
      * 保持输入输出size一致  
      * 让输入特征保留更多信息
    * padding=1 补一圈的0 =2 补2圈的0
    * ![](images/convolution-2022-09-28-11-07-47.png)

## pytorch中的卷积
`
# Conv2d类   
```
class torch.nn.Conv2d(in_channels, #输入特征图的通道数
                      out_channels, #输出特征图的通道数
                      kernel_size, # 卷积核大小
                      stride=1, #步长 default=1
                      padding=0, #when padding='valid'(没有padding) or 'same'(让输出特征图和输入特征图大小一致) stride must=1
                      整数时，表示特征图外边补多少圈0
                      tuple时，表示在特征图的行/列补多少零
                      dilation=1, 
                      groups=1, 
                      bias=True, 
                      padding_mode='zeros', 
                      device=None, 
                      dtype=None)
```


In [8]:
import re
import torch
import torch.nn as nn

input_feat = torch.tensor([[4, 1, 7, 5], [4, 4, 2, 5], [7, 7, 2, 4], [1, 0, 2, 4]], dtype=torch.float32)
print(input_feat)
print(input_feat.shape)

conv2d = nn.Conv2d(1, 1, (2,2),stride=1, padding='same',bias=True)
print(conv2d.weight)
print(conv2d.bias)

# 卷积核要有四个维度(输入通道数，输出通道数，高，宽)
kernels = torch.tensor([[[[1, 0], [2, 1]]]], dtype=torch.float32)
conv2d.weight = nn.Parameter(kernels, requires_grad=False)

print(conv2d.weight)
print(conv2d.bias)

#pytorch 输入tensor的维度信息是 (batch_size, 通道数，宽，高)
# output = conv2d(input_feat)
# print(output)
input_feat1 =  input_feat.unsqueeze(0).unsqueeze(0)

print(input_feat1)
print(input_feat1.shape)


output = conv2d(input_feat1)
print(output)

tensor([[4., 1., 7., 5.],
        [4., 4., 2., 5.],
        [7., 7., 2., 4.],
        [1., 0., 2., 4.]])
torch.Size([4, 4])
Parameter containing:
tensor([[[[ 0.0770,  0.4122],
          [-0.4616,  0.0716]]]], requires_grad=True)
Parameter containing:
tensor([-0.2050], requires_grad=True)
Parameter containing:
tensor([[[[1., 0.],
          [2., 1.]]]])
Parameter containing:
tensor([-0.2050], requires_grad=True)
tensor([[[[4., 1., 7., 5.],
          [4., 4., 2., 5.],
          [7., 7., 2., 4.],
          [1., 0., 2., 4.]]]])
torch.Size([1, 1, 4, 4])
tensor([[[[15.7950, 10.7950, 15.7950, 14.7950],
          [24.7950, 19.7950,  9.7950, 12.7950],
          [ 8.7950,  8.7950,  9.7950, 11.7950],
          [ 0.7950, -0.2050,  1.7950,  3.7950]]]],
       grad_fn=<ConvolutionBackward0>)


# 卷积的变种
|卷积|说明|
|---|---|
|深度可分离卷积|用于轻量化模型|
|空洞卷积|图像分割|
|转置卷积|图像分割|
|残差卷积|为了提高网络精度的一种组合|
|inception模块|为了提高网络精度的一种组合|
|SE块|为了提高网络精度的一种组合|

## 深度可分离卷积
* DW 卷积 ，
  * 标准卷积计算
  ![](images/convolution-2022-09-28-15-58-09.png)   
  * DW depthwise卷积
    * 有m个卷积核的卷积，每个卷积核的通道数为1，输出为m个通道的特征图
    * ![](images/convolution-2022-09-28-16-03-06.png)
* PW pointwise卷积
  * 就是标准卷积，不过卷积核为1x1
  * 逐点卷积，将DW输出的m个特征图，输出一个n通道的特征图
  * 实现方式为n个卷积核为1x1的卷积，每个卷积核的通道数为m
  * ![](images/convolution-2022-09-28-16-06-16.png)
  * 可以获得一个与标准卷积相同尺寸的结果
* 计算量
  * 标准卷积
    * `k*k*m*n*h'*w'`
  * 可分离卷积
    * `k*k*m*h'*w'+1*1*m*n*h'*w'`
  * 二者之比为
    * `1/n+1/(k*k)`


# pytorch 实现深度可分离
```
 torch.nn.Conv2d(in_channels, 
 out_channels, 
 kernel_size, 
 stride=1, 
 padding=0, 
 dilation=1, 
 groups=1, 
 bias=True, 
 padding_mode='zeros', 
 device=None, 
 dtype=None)
```
* groups =1 为标准卷积
* groups 不等于1时，输入特征图会分成groups组，每组有自己的卷积核，最后输出的特征图有groups个分组，必须能整除in_channels,out_channels
* groups = in_channels时，即DW卷积
* Attention
  * DW中，输入特征通道数和输出通道数一致
  * 一般DW卷积核3x3
  * DW卷积的groups参数和输出通道数一致

In [10]:
import torch 
import torch.nn as nn
#[batchsize, channel,width,height]
x =torch.rand((3,5,5)).unsqueeze(0)
print(x.shape)

#DW中，输入特征通道数和输出通道数一致
in_channels_dw = x.shape[1]
out_channels_dw = x.shape[1]

#一般DW卷积核大小3
kernel_size =3 
stride =1
dw = nn.Conv2d(in_channels_dw, out_channels_dw,kernel_size,
stride,
groups = in_channels_dw
)


in_channels_pw = out_channels_dw
out_channels_pw = 4
kernel_size_pw = 1
pw = nn.Conv2d(in_channels_pw, out_channels_pw, kernel_size_pw, stride)
out = pw(dw(x))
print(out.shape)

torch.Size([1, 3, 5, 5])
torch.Size([1, 4, 3, 3])


## 空洞卷积
* 常用于图像分割中(对每个像素点都进行预测)
* 图像分割模型，会采用多层卷积来提取特征，随着层数的不断加深，感受野也越来越大，但是对于图像分割模型，经过多层的卷积和pooling后，特征图会变小。为了做到每个像素点都有预测输出，需要对较小的特征图进行上采样或反卷积，将特征图扩大到一定尺度，然后再进行预测。
* 从一个较小的特征图恢复到较大的特征图，会带来一定的信息损失，所以为了保证有较大的感受野，同时又不用缩小特征图-空洞卷积
* 卷积中的pool层(使特征图变小)以及卷积层，使得特征图越来越小，不同层的特征图的计算区域不同-感受野(卷积输出时一个单元在输入特征图中代表的单元数)。感受野越大标识包含的信息更加全面，语义信息更加抽象，越小，代表这个包含更加细节的语义信息。
* 计算过程
  * 将卷积核以一定比例拆分开
  * 标准卷积
    * ![](images/convolution-2022-09-28-21-00-18.png)
  * 空洞卷积
    * ![](images/convolution-2022-09-28-21-00-45.png)

In [12]:
import torch 
import torch.nn as nn
#[batchsize, channel,width,height]
x =torch.rand((3,128,128)).unsqueeze(0)
print(x.shape)


in_channels_dw = x.shape[1]
out_channels_dw = x.shape[1]

#一般DW卷积核大小3
kernel_size =3 
stride =1
dw = nn.Conv2d(in_channels_dw, out_channels_dw, kernel_size,
stride,
groups = in_channels_dw
)

in_channels_pw = out_channels_dw
out_channels_pw = 10
pw = nn.Conv2d(in_channels_pw, out_channels_pw, 1)
output = pw(dw(x))
print(output.shape)

torch.Size([1, 3, 128, 128])
torch.Size([1, 10, 126, 126])
